# LAB: Neural Network in Keras

NOTE: This notebook should be run in AWS SageMaker Tensorflow environment.

Import dependencies

In [ ]:
from numpy import loadtxt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report

Load `pima-indians-diabetes.csv` dataset

In [ ]:
# This dataset can be found on Kaggle as well: https://www.kaggle.com/datasets/yasserh/breast-cancer-dataset

import os

DATASET = 'pima-indians-diabetes.csv'
DATA_FOLDER = 's3://bah-data'

data = pd.read_csv(os.path.join(DATA_FOLDER, DATASET), header=None)

Check your dataset. See if you need to prepare it for further modeling.

Do some analysis, sanity checks etc.

In [ ]:
data.shape

(768, 9)

In [ ]:
data.head(5)

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
data.isnull().any()

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
dtype: bool

In [ ]:
data[8].value_counts()

0    500
1    268
Name: 8, dtype: int64

Split train test dataset

In [ ]:
X = data.iloc[:, :len(data.columns)-1]
y = data.iloc[:, len(data.columns)-1]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

Use `Sequential` class from Keras to construct Feed Forward Neural Network with:
    
- Input layer (pay attention to `input_shape` parameter)
- Two hidden layers:
    - First one with 64 dimension
    - Second one with 32 dimension
    - Third one with 16 dimension
    - Use `elu` activation function in both of these hidden layers
- Output layer with `sigmoid` function

In [ ]:
model = Sequential()
model.add(Dense(64, input_shape=(len(data.columns)-1,), activation='elu'))
model.add(Dense(31, activation='elu'))
model.add(Dense(16, activation='elu'))
model.add(Dense(1, activation='sigmoid'))

2023-03-17 14:39:26.645267: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Compile the model

Pay attention to:

- learning rate
- batch size
- epoch size
- loss function
- optimizer

In [ ]:
LEARNING_RATE = 0.001
BATCH = 10
EPOCH = 50

# Set loss function to be used
loss_function = keras.losses.BinaryCrossentropy(from_logits=True)

# Set optimizer to be used. 
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)

In [ ]:
model.compile(loss=loss_function,
              optimizer=optimizer, 
              metrics=['accuracy'])

Fit the model

In [ ]:
model.fit(X_train, y_train, epochs=EPOCH, batch_size=BATCH, verbose=0)

/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/keras/backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Predict on test dataset

In [ ]:
yhat = model.predict(X_test)

6/6 [==============================] - 0s 2ms/step


Evaluate the model on test dataset results

In [ ]:
print(classification_report(y_test, np.round(yhat)))

              precision    recall  f1-score   support

           0       0.77      0.82      0.79       123
           1       0.63      0.55      0.59        69

    accuracy                           0.72       192
   macro avg       0.70      0.69      0.69       192
weighted avg       0.72      0.72      0.72       192

